In [1]:
!nvidia-smi

Wed Aug 31 08:26:16 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P0    30W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
!cp /content/gdrive/MyDrive/celeba_hq.zip ./
!unzip ./celeba_hq.zip -d ./

Streaming output truncated to the last 5000 lines.
  inflating: ./celeba_hq/train/female/174763.jpg  
  inflating: ./celeba_hq/train/female/174787.jpg  
  inflating: ./celeba_hq/train/female/174791.jpg  
  inflating: ./celeba_hq/train/female/174799.jpg  
  inflating: ./celeba_hq/train/female/174833.jpg  
  inflating: ./celeba_hq/train/female/174834.jpg  
  inflating: ./celeba_hq/train/female/174839.jpg  
  inflating: ./celeba_hq/train/female/174861.jpg  
  inflating: ./celeba_hq/train/female/174868.jpg  
  inflating: ./celeba_hq/train/female/174874.jpg  
  inflating: ./celeba_hq/train/female/174881.jpg  
  inflating: ./celeba_hq/train/female/174889.jpg  
  inflating: ./celeba_hq/train/female/174890.jpg  
  inflating: ./celeba_hq/train/female/174891.jpg  
  inflating: ./celeba_hq/train/female/174921.jpg  
  inflating: ./celeba_hq/train/female/174924.jpg  
  inflating: ./celeba_hq/train/female/174943.jpg  
  inflating: ./celeba_hq/train/female/174962.jpg  
  inflating: ./celeba_hq/train/

In [4]:
!mv /content/celeba_hq/train/female/* /content/celeba_hq/train
!mv /content/celeba_hq/train/male/* /content/celeba_hq/train
!rm -rf /content/celeba_hq/train/female
!rm -rf /content/celeba_hq/train/male

In [ ]:
!mkdir ./checkpoints/experiment_name

mkdir: cannot create directory ‘./checkpoints/experiment_name’: No such file or directory


In [ ]:
!rm ./checkpoints/experiment_name/*

In [ ]:
!cp -r /content/gdrive/MyDrive/Pluralistic-Inpainting/checkpoints/experiment_name_2/* ./checkpoints/experiment_name/

In [5]:
!rm -rf ./checkpoints/experiment_name
!mkdir ./checkpoints

In [ ]:
!rm -rf ./checkpoints
!rm -rf ./runs

In [ ]:
!rm -rf /content/gdrive/MyDrive/Pluralistic-Inpainting/checkpoints/experiment_name

In [ ]:
!ln -s /content/gdrive/MyDrive/Pluralistic-Inpainting/checkpoints/experiment_name ./checkpoints/experiment_name

In [6]:
!ln -s /content/gdrive/MyDrive/Pluralistic-Inpainting/runs runs
!ln -s /content/gdrive/MyDrive/Pluralistic-Inpainting/checkpoints/experiment_name ./checkpoints/experiment_name

In [ ]:
!rm -rf ./runs

In [7]:
! npm install -g localtunnel
get_ipython().system_raw('python3 -m pip install visdom')
get_ipython().system_raw('python3 -m visdom.server -port 8097 >> visdomlog.txt 2>&1 &')   
get_ipython().system_raw('lt --port 8097 >> url.txt 2>&1 &')   
import time
!pip install dominate
! cat url.txt
import sys
sys.path.append('/content/gdrive/MyDrive/Pluralistic-Inpainting')
from options.train_options import TrainOptions
from dataloader.data_loader import dataloader
from model import create_model
from util.visualizer import Visualizer
from torch.utils.tensorboard import SummaryWriter

writer = SummaryWriter()


/tools/node/bin/lt -> /tools/node/lib/node_modules/localtunnel/bin/lt.js
+ localtunnel@2.0.2
added 22 packages from 22 contributors in 1.811s
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
your url is: https://beige-jeans-win-34-83-119-78.loca.lt


In [ ]:
%load_ext tensorboard
%tensorboard --logdir /content/runs/Aug29_10-35-20_1ec52db2a425

In [ ]:
if __name__ == '__main__':
    
    # get training options
    sys.argv = ['train.ipynb']
    opt = TrainOptions().parse()
    opt.continue_train = False
    opt.niter = 100000
    # create a dataset
    dataset = dataloader(opt)
    dataset_size = len(dataset) * opt.batchSize
    print('training images = %d' % dataset_size)
    # create a model
    model = create_model(opt)
    # create a visualizer
    visualizer = Visualizer(opt)
    # training flag
    keep_training = True
    max_iteration = opt.niter+opt.niter_decay
    epoch = 0
    total_iteration = opt.iter_count

    # training process
    while(keep_training):
        epoch_start_time = time.time()
        epoch+=1
        print('\n Training epoch: %d' % epoch)

        for i, data in enumerate(dataset):
            iter_start_time = time.time()
            total_iteration += 1
            model.set_input(data)
            model.optimize_parameters()

            # display images on visdom and save images
            if total_iteration % opt.display_freq == 0:
                visualizer.display_current_results(model.get_current_visuals(), epoch)
                writer.add_scalars('Loss', model.get_current_errors(), total_iteration)
                visualizer.plot_current_distribution(model.get_current_dis())

            # print training loss and save logging information to the disk
            if total_iteration % opt.print_freq == 0:
                losses = model.get_current_errors()
                t = (time.time() - iter_start_time) / opt.batchSize
                visualizer.print_current_errors(epoch, total_iteration, losses, t)
                if opt.display_id > 0:
                    visualizer.plot_current_errors(total_iteration, losses)

            # save the latest model every <save_latest_freq> iterations to the disk
            if total_iteration % opt.save_latest_freq == 0:
                print('saving the latest model (epoch %d, total_steps %d)' % (epoch, total_iteration))
                model.save_networks('latest')

            # save the model every <save_iter_freq> iterations to the disk
            if total_iteration % opt.save_iters_freq == 0:
                print('saving the model of iterations %d' % total_iteration)
                model.save_networks(total_iteration)

            if total_iteration > max_iteration:
                keep_training = False
                break

        model.update_learning_rate()

        print('\nEnd training')


In [ ]:
!cp -r /content/runs/*  /content/gdrive/MyDrive/Pluralistic-Inpainting/runs/
!cp /content/checkpoints/experiment_name/latest* /content/gdrive/MyDrive/Pluralistic-Inpainting/checkpoints/

In [ ]:
!cp /content/gdrive/MyDrive/Pluralistic-Inpainting/checkpoints/latest* ./checkpoints/

In [ ]:
!mv /content/celeba_hq/val/female/* /content/celeba_hq/val
!mv /content/celeba_hq/val/male/* /content/celeba_hq/val
!rm -rf /content/celeba_hq/val/female
!rm -rf /content/celeba_hq/val/male

In [ ]:
import sys
sys.path.append('/content/gdrive/MyDrive/Pluralistic-Inpainting')
!pip install dominate
from options import test_options
from dataloader import data_loader
from model import create_model
from util import visualizer
from itertools import islice

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!rm -rf ./results
!mkdir ./results

In [ ]:
if __name__=='__main__':
    # get testing options
    sys.argv = ['train.ipynb']
    opt = test_options.TestOptions().parse()
    opt.img_file = '/content/celeba_hq/val'
    opt.nsampling = 1
    opt.batchSize = 2
    # creat a dataset
    dataset = data_loader.dataloader(opt)
    dataset_size = len(dataset) * opt.batchSize
    print('testing images = %d' % dataset_size)
    # create a model
    model = create_model(opt)
    model.eval()


    for i, data in enumerate(islice(dataset, 2000)):
        model.set_input(data)
        model.test()


In [ ]:
!mv /content/experiment_name/latest* 

In [ ]:
ls -l ./images/out0/ | grep "^-" | wc -l

2000


In [ ]:
!python /content/gdrive/MyDrive/Pluralistic-Inpainting/classify_photos.py

In [ ]:
!python /content/gdrive/MyDrive/Pluralistic-Inpainting/classify_for_lpips.py

In [ ]:
!python /content/gdrive/MyDrive/Pluralistic-Inpainting/evaluations/fid_score.py ./images/truth ./images/out

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
Downloading: "https://github.com/mseitzer/pytorch-fid/releases/download/fid_weights/pt_inception-2015-12-05-6726825d.pth" to /root/.cache/torch/hub/checkpoints/pt_inception-2015-12-05-6726825d.pth
100% 91.2M/91.2M [00:02<00:00, 36.5MB/s]
100% 40/40 [00:08<00:00,  4.66it/s]
100% 40/40 [00:07<00:00,  5.21it/s]
FID:  13.788708925620313


In [ ]:
!mv /content/checkpoints/experiment_name/190000* ./checkpoints/experiment_name/

In [ ]:
!rm -rf ./images/*

In [ ]:
!mkdir ./images

mkdir: cannot create directory ‘./images’: File exists


In [ ]:
!rm -rf ./results/*

In [ ]:
import sys
!pip install lpips
sys.path.append('/content/gdrive/MyDrive/Pluralistic-Inpainting')
!python /content/gdrive/MyDrive/Pluralistic-Inpainting/evaluations/lpips_score.py

In [ ]:
import sys
sys.path.append('/content/gdrive/MyDrive/Pluralistic-Inpainting')
!python /content/gdrive/MyDrive/Pluralistic-Inpainting/util/evaluation.py

In [ ]:
!rm /content/checkpoints/experiment_name/latest*
!cp /content/gdrive/MyDrive/Pluralistic-Inpainting/checkpoints/experiment_name_1/* /content/checkpoints/experiment_name/

In [ ]:
!rm /content/checkpoints/experiment_name/latest*

In [ ]:
!mv /content/gdrive/MyDrive/Pluralistic-Inpainting/latest* /content/checkpoints/experiment_name/

In [ ]:
import sys
sys.path.append('/content/gdrive/MyDrive/Pluralistic-Inpainting')
!pip install dominate
from options import PD_train_options, test_options
from dataloader import data_loader
from model import create_model
from util import visualizer
import time

if __name__=='__main__':
    sys.argv = ['train.ipynb']
    # get testing options
    opt_feed = test_options.TestOptions().parse()
    opt_feed.batchSize = 6
    # creat a dataset
    opt_feed.img_file = '/content/celeba_hq/train'
    dataset = data_loader.dataloader(opt_feed)
    dataset_size = len(dataset) * opt_feed.batchSize
    # create models
    model_pic = create_model(opt_feed)
    model_pic.eval()
    opt = PD_train_options.PD_trainOptions().parse()
    opt.continue_train = True
    opt.batchSize = 6
    opt.img_file = '/content/celeba_hq/train'
    opt.model = 'pdgan'
    model_pd = create_model(opt)
    model_pd.eval()
    # create a visualizer
    visualizer = visualizer.Visualizer(opt)
    keep_training = True
    epoch = 7
    max_iteration = opt.niter+opt.niter_decay
    total_iteration = 33700

    while(keep_training):
        epoch_start_time = time.time()
        epoch += 1
        print('\n Training epoch: %d' % epoch)

        for i, data in enumerate(dataset):
            iter_start_time = time.time()
            total_iteration += 1
            model_pic.set_input(data)
            img_p, mask, img_truth = model_pic.feed()
            model_pd.set_input(data)
            opt.lambda_rec = 0.1
            opt.use_gated = False
            model_pd.pd_optimize_parameters(img_p)

            # display images on visdom and save images
            if total_iteration % opt.display_freq == 0:
                visualizer.display_current_results(model_pd.get_current_visuals(), epoch)
                writer.add_scalars('Loss', model_pd.get_current_errors(), total_iteration)

            # print training loss and save logging information to the disk
            if total_iteration % opt.print_freq == 0:
                losses = model_pd.get_current_errors()
                t = (time.time() - iter_start_time) / opt.batchSize
                visualizer.print_current_errors(epoch, total_iteration, losses, t)
                if opt.display_id > 0:
                    visualizer.plot_current_errors(total_iteration, losses)

            # save the latest model_pd every <save_latest_freq> iterations to the disk
            if total_iteration % opt.save_latest_freq == 0:
                print('saving the latest model_pd (epoch %d, total_steps %d)' % (epoch, total_iteration))
                model_pd.save_networks('latest')

            # save the model_pd every <save_iter_freq> iterations to the disk
            if total_iteration % opt.save_iters_freq == 0:
                print('saving the model_pd of iterations %d' % total_iteration)
                model_pd.save_networks(total_iteration)

            if total_iteration > max_iteration:
                keep_training = False
                break

        model_pd.update_learning_rate()



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
--------------Options--------------
batchSize: 2
checkpoints_dir: ./checkpoints
display_id: 1
display_port: 8097
display_single_pane_ncols: 0
display_winsize: 256
fineSize: [256, 256]
gpu_ids: 0
how_many: 500
img_file: /content/celeba_hq/train
isTrain: False
loadSize: [266, 266]
mask_file: none
mask_type: [1, 2]
model: pluralistic
nThreads: 8
name: experiment_name
no_augment: False
no_flip: False
no_rotation: False
no_shuffle: False
nsampling: 1
ntest: inf
output_scale: 4
pd_isTrain: None
phase: test
resize_or_crop: resize_and_crop
results_dir: ./results/
save_number: 10
which_iter: latest
----------------End----------------


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:566: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


ResEncoder(
  (block0): ResBlockEncoderOptimized(
    (conv1): SpectralNorm(
      (module): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    )
    (conv2): SpectralNorm(
      (module): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    )
    (bypass): SpectralNorm(
      (module): Conv2d(3, 32, kernel_size=(1, 1), stride=(1, 1))
    )
    (model): Sequential(
      (0): SpectralNorm(
        (module): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      )
      (1): LeakyReLU(negative_slope=0.1)
      (2): SpectralNorm(
        (module): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      )
      (3): AvgPool2d(kernel_size=2, stride=2, padding=0)
    )
    (shortcut): Sequential(
      (0): AvgPool2d(kernel_size=2, stride=2, padding=0)
      (1): SpectralNorm(
        (module): Conv2d(3, 32, kernel_size=(1, 1), stride=(1, 1))
      )
    )
  )
  (encoder0): ResBlock(
    (pool): AvgPool2d(kernel_size=2, stri

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth


  0%|          | 0.00/528M [00:00<?, ?B/s]

model [pdgan] was created

 Training epoch: 8
(epoch: 8, iters: 33800, time: 0.285) app_g: 10.077 ad_g: 1.100 img_d: 0.302 pd: 1.614 tv: 0.557 
(epoch: 8, iters: 33900, time: 0.276) app_g: 10.730 ad_g: 1.239 img_d: 0.255 pd: 1.593 tv: 0.498 
(epoch: 8, iters: 34000, time: 0.279) app_g: 12.161 ad_g: 1.417 img_d: 0.303 pd: 1.586 tv: 0.465 
saving the latest model_pd (epoch 8, total_steps 34000)
(epoch: 8, iters: 34100, time: 0.268) app_g: 12.323 ad_g: 0.993 img_d: 0.263 pd: 1.193 tv: 0.619 
(epoch: 8, iters: 34200, time: 0.279) app_g: 11.654 ad_g: 1.229 img_d: 0.184 pd: 1.862 tv: 0.569 
(epoch: 8, iters: 34300, time: 0.275) app_g: 11.503 ad_g: 0.922 img_d: 0.332 pd: 1.504 tv: 0.604 
(epoch: 8, iters: 34400, time: 0.274) app_g: 11.211 ad_g: 1.525 img_d: 0.275 pd: 0.811 tv: 0.712 
(epoch: 8, iters: 34500, time: 0.277) app_g: 11.679 ad_g: 1.369 img_d: 0.270 pd: 1.243 tv: 0.665 
(epoch: 8, iters: 34600, time: 0.274) app_g: 14.525 ad_g: 1.292 img_d: 0.289 pd: 1.226 tv: 0.641 
(epoch: 8, iters

FileNotFoundError: ignored

In [4]:
!rm -rf /content/runs/Aug29*

In [3]:
!apt install psmisc

Reading package lists... Done
Building dependency tree       
Reading state information... Done
psmisc is already the newest version (23.1-1ubuntu0.1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 20 not upgraded.


In [5]:
!sudo fuser /dev/nvidia*

Cannot stat file /proc/691/fd/57: Transport endpoint is not connected
Cannot stat file /proc/691/fd/111: Transport endpoint is not connected
Cannot stat file /proc/691/fd/112: Transport endpoint is not connected
Cannot stat file /proc/691/fd/113: Transport endpoint is not connected
Cannot stat file /proc/691/fd/114: Transport endpoint is not connected
Cannot stat file /proc/691/fd/115: Transport endpoint is not connected
/dev/nvidia0:          691m
/dev/nvidiactl:        691m
/dev/nvidia-uvm:       691m


In [ ]:
!kill -9 691

In [ ]:
import sys
sys.path.append('/content/gdrive/MyDrive/Pluralistic-Inpainting')
!pip install dominate
from options import PD_test_options, test_options
from dataloader import data_loader
from model import create_model
from itertools import islice

if __name__ == "__main__":
    # get testing options
    sys.argv = ['train.ipynb']
    opt_feed = test_options.TestOptions().parse()
    opt_feed.img_file = '/content/celeba_hq/val'
    # creat a dataset
    opt_feed.nsampling = 1
    dataset = data_loader.dataloader(opt_feed)
    dataset_size = len(dataset) * opt_feed.batchSize
    # create models
    model_pic = create_model(opt_feed)
    model_pic.eval()
    opt = PD_test_options.PD_TestOptions().parse()
    opt.nsampling = 1
    opt.img_file = '/content/celeba_hq/val'
    # creat a dataset
    dataset = data_loader.dataloader(opt)
    dataset_size = len(dataset) * opt.batchSize
    print("testing images = %d" % dataset_size)
    # create a model
    model = create_model(opt)
    model.eval()
    for i, data in enumerate(islice(dataset, 1000)):
        model_pic.set_input(data)
        img_ps = []
        for _ in range(opt_feed.nsampling):
            img_p, mask, img_truth = model_pic.feed()
            img_ps.append(img_p)
        model.set_input(data)
        num = 0
        for img_p in img_ps:
            num = model.test(img_p, num)

In [ ]:
!rm ./results/*
!rm -rf ./images/*

rm: cannot remove './results/*': No such file or directory
